In [2]:
import os
import tensorflow as tf
import json
import pandas as pd
from os import getenv
#from keras import models
from firebase_admin import credentials, firestore, initialize_app

In [3]:
cred = credentials.Certificate("key.json")
default_app = initialize_app(cred)
db = firestore.client()

In [4]:
class User(object):
    def __init__(self, USER_ID, Nama, Gender, Age, Tipe_Masalah):
        self.USER_ID = USER_ID
        self.Nama = Nama
        self.Gender = Gender
        self.Age = Age
        self.Tipe_Masalah = Tipe_Masalah

    @staticmethod
    def from_json_do_not_use(source):
        """
        Source is expected from pure json string
        """
        return source

    @staticmethod
    def from_dict(source):
        # [START_EXCLUDE]
        user = User(source[u'USER_ID'], source[u'Nama'], source[u'Gender'], source[u'Age'], source[u'Tipe_Masalah'])

        if u'USER_ID' in source:
            user.USER_ID = source[u'USER_ID']

        if u'Nama' in source:
            user.Nama = source[u'Nama']

        if u'Gender' in source:
            user.Gender = source[u'Gender']

        if u'Age' in source:
            user.Age = source[u'Age']

        if u'Tipe_Masalah' in source:
            user.Tipe_Masalah = source[u'Tipe_Masalah']

        return user
        # [END_EXCLUDE]

    def to_dict(self):
        # [START_EXCLUDE]
        dest = {
            u'USER_ID': self.USER_ID,
            u'Nama': self.Nama,
            u'Gender': self.Gender,
            u'Age': self.Age,
            u'Tipe_Masalah': self.Tipe_Masalah
        }

        if self.USER_ID:
            dest[u'USER_ID'] = self.USER_ID

        if self.Nama:
            dest[u'Nama'] = self.Nama

        if self.Gender:
            dest[u'Gender'] = self.Gender

        if self.Age:
            dest[u'Age'] = self.Age

        if self.Tipe_Masalah:
            dest[u'Tipe_Masalah'] = self.Tipe_Masalah

        return dest
        # [END_EXCLUDE]

    
    def __repr__(self):
        return (
            f'City(\
                USER_ID={self.USER_ID}, \
                Nama={self.Nama}, \
                Age={self.Age}, \
                Gender={self.Gender}, \
                Tipe_Masalah={self.Tipe_Masalah}\
            )'
        )

In [5]:
zidan = User(1, "zidan", "pria", 43, "Depresi")

In [6]:
zidan

City(                USER_ID=1,                 Nama=zidan,                 Age=43,                 Gender=pria,                 Tipe_Masalah=Depresi            )

In [7]:
type(zidan)

__main__.User

In [8]:
type(zidan.to_dict())

dict

In [8]:
import pathlib
import os

user_file = "\\Data\\Processed User Data.json"
caregiver_file = "\\Data\\Processed Caregiver Data.json"

path = pathlib.Path().resolve()
parent_path = path.parent.absolute()

user_path = str(parent_path) + user_file
caregiver_ = str(parent_path) + caregiver_file

In [9]:
"""
with open(user_path) as file:
    data = json.load(file)
    db_user_placeholder = db.collection(u'User_Data_Placeholder')
    for user_dict in data:
        user_dict['Nama'] = 'placeholder_nama'
        user_dict['Text'] = 'Placeholder_text'
        db_user_placeholder.add(user_dict)
"""

In [11]:
data = db.collection(u'user_pref').get()


In [23]:
print(data[0].to_dict())

{'gender': 'Laki-laki', 'age': 27, 'text': 'Placeholder_text', 'problems': 'Gangguan-kecemasan Depresi', 'name': 'dhiva'}


In [24]:
def convert_snapshot(data, desired_key):
    """
    this extract desired keys from list of dicts from snapshot
    """
    data_dictionary = map(lambda x: x.to_dict(), data)
    x = map(lambda x: dict((k, x[k]) for k in desired_key if k in x) , list(data_dictionary))
    
    return list(x)

In [25]:
new_data = convert_snapshot(data, ["gender","problems","age","text"])

In [26]:
print(new_data)

[{'gender': 'Laki-laki', 'problems': 'Gangguan-kecemasan Depresi', 'age': 27, 'text': 'Placeholder_text'}]


In [27]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    """
    A utility method to create a tf.data dataset from a Pandas Dataframe
    """
    dataframe = dataframe.copy()
    ds = tf.data.Dataset.from_tensor_slices(dict(dataframe))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [35]:
def convert_categorical_data(df, col, is_caregiver = False):
    """
    Convert categorical data that splited by space to their own column in pandas dataframe , making them like one hot encoded
    """
    import numpy as np
    ### Join every string in every row, split the result, pull out the unique values.
    genres = np.unique(' '.join(df[col]).split(' '))
    ### Drop 'NA'
    genres = np.delete(genres, np.where(genres == ''))
    if(not is_caregiver):
        for genre in genres:
            df[genre] = df[col].str.contains(genre).astype('int')
    else:
        for genre in genres:
            df['Caregiver-'+genre] = df[col].str.contains(genre).astype('int')
    df.drop(col, axis=1, inplace=True)

In [58]:
def fill_non_existent_column(df, new_column=None):
    """
    This fill the df with unique problems column i
    """
    if new_column is None:
         new_column = ['ADHD-Hiperaktif-dan-kurang-fokus', 'Depresi', 'Gangguan-kecemasan', 'Gangguan-makan','Gangguan-stres-pascatrauma', 'Skizofrenia']
    for col in new_column:
        if(col in df.columns):
            pass
        df[col] = 0
    
    

In [45]:
def change_column_name(df, old_name,new_name):
    """
    replace list of column name in old_name to new colum name in new_name
    the order of old_name and new_name need to be exact
    """
    res = {old_name[i]: new_name[i] for i in range(len(new_name))}
    df.rename(columns = res, inplace = True)
    
    

In [36]:
#ds_kalimat = tf.data.Dataset.from_tensor_slices(pd.DataFrame.from_dict(new_data).to_dict(orient="list"))

In [46]:
user_df = pd.DataFrame.from_dict(new_data)

In [47]:
user_df.head(4)

,gender,problems,age,text
0,Laki-laki,Gangguan-kecemasan Depresi,27,Placeholder_text


In [49]:
convert_categorical_data(user_df, col='problems')

In [50]:
user_df.head(4)

,gender,age,text,Depresi,Gangguan-kecemasan
0,Laki-laki,27,Placeholder_text,1,1


In [51]:
change_column_name(user_df, ['gender','age'], ['Gender','Age'])

In [52]:
user_df.head(4)

,Gender,Age,text,Depresi,Gangguan-kecemasan
0,Laki-laki,27,Placeholder_text,1,1


In [59]:
user_df.columns

Index(['Gender', 'Age', 'text', 'Depresi', 'Gangguan-kecemasan'], dtype='object')

In [60]:
 fill_non_existent_column(user_df)

In [61]:
user_df.head(4)


,Gender,Age,text,Depresi,Gangguan-kecemasan,ADHD-Hiperaktif-dan-kurang-fokus,Gangguan-makan,Gangguan-stres-pascatrauma,Skizofrenia
0,Laki-laki,27,Placeholder_text,0,0,0,0,0,0


In [81]:
user_ds = df_to_dataset(user_df)

<BatchDataset element_spec={'Gender': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'Age': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'text': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'Depresi': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Gangguan-kecemasan': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'ADHD-Hiperaktif-dan-kurang-fokus': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Gangguan-makan': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Gangguan-stres-pascatrauma': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Skizofrenia': TensorSpec(shape=(None,), dtype=tf.int64, name=None)}>

In [ ]:
#caregiver_query

In [64]:
data = db.collection(u'chat_rooms').get()

In [71]:
test_data = data[0].to_dict()

In [72]:
test_data_caregiver = test_Data['caregiver'].get()

In [74]:
test_data_caregiver.to_dict()

{'name': 'dhiva',
 'gender': 'Laki-laki',
 'point': 0,
 'birthday': DatetimeWithNanoseconds(2000, 11, 16, 16, 0, tzinfo=datetime.timezone.utc)}

In [75]:
test_data

{'problems': 'Depresi ADHD-Hiperaktif-dan-kurang-fokus',
 'users': ['i5riAITSRidbvwTRZxBwnbiB0A52', 'i5riAITSRidbvwTRZxBwnbiB0A52'],
 'text': 'placeholder text',
 'max_slot': 3,
 'caregiver': <google.cloud.firestore_v1.document.DocumentReference at 0x1f07bd25f40>,
 'is_open': True}

In [76]:
from datetime import date
 
def age(birthdate):
    today = date.today()
    age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
    return age

In [77]:
age(test_data_caregiver.to_dict()['birthday'])

21

In [80]:
import flatdict

In [ ]:

# Python code to merge dict using update() method
def Merge(dict1, dict2):
    return(dict2.update(dict1))

def unpack_caregiver_dict(dictionary):
    

In [109]:
caregiver_dict_unpacked = map(lambda feature: feature.to_dict() , data)

In [110]:
caregiver_dict = map(lambda feature: feature.update(feature.pop('caregiver').get().to_dict()) , caregiver_dict_unpacked )

In [111]:
caregiver_dict = list(caregiver_dict)

In [112]:
print(caregiver_dict)

[None, None]
